In [1]:
!pip install opendatasets
!pip install pandas

^C



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105798 sha256=d930c92fd2c3e8975ade8efea5c93b62c55061fc2dfe55b9e24e29aed593d7e1
  Stored in directory: c:\users\omid\appdata\local\pip\cache\wheels\46\d2\26\84d0a1acdb9c6baccf7d28cf06962ec80529fe1ad938489983
Successfully built kaggle



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import opendatasets as od
import pandas
 
od.download(
    "https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset")

In [ ]:
import pandas as pd

data = pd.read_csv('movielens-20m-dataset/rating.csv')
data['userId'] -= 1
data = data.drop(columns = ['timestamp'])
data.head()

In [ ]:
import numpy as np

total_movie = set(data['movieId'])
movie2idx = {}
counter = 0
for movieId in total_movie:
  movie2idx[movieId] = counter
  counter += 1
data.head()
raw_data = data.to_numpy()

for row in range(len(raw_data)):
  raw_data[row, 1] = movie2idx[raw_data[row, 1]]

In [ ]:
stars = set(data['rating'])
star2idx = {}
counter = 1
for star in stars:
  star2idx[star] = counter
  counter += 1
for row in range(len(stars)):
  raw_data[row, 2] = star2idx[raw_data[row, 2]]
raw_data = raw_data.astype(int)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, _, _ = train_test_split(raw_data, raw_data, test_size=0.2, random_state=42)

In [ ]:
from scipy.sparse import lil_matrix

sparse_matrix_train = lil_matrix((max(raw_data[:,0]) + 1, max(raw_data[:,1]) + 1))
sparse_matrix_test = lil_matrix((max(raw_data[:,0]) + 1, max(raw_data[:,1]) + 1))

for row in X_train:
  sparse_matrix_train[row[0], row[1]] = row[2]
for row in X_test:
  sparse_matrix_test[row[0], row[1]] = row[2]

In [ ]:
def one_hard_encode(X, K):
    N, D = X.shape
    Y = np.zeros((N, D, K))
    for n, d in zip(*X.nonzero()):
        Y[n, d, X[n, d]] = 1
    return Y

In [ ]:
def one_hard_mask_encode(X, K):
    N, D = X.shape
    Y = np.zeros((N, D, K))
    for n, d in zip(*X.nonzero()):
        Y[n, d, :] = 1
    return Y

In [ ]:
def prob2rate(prob):
  return prob.dot((np.arange(10) + 1))

In [ ]:
import tensorflow as tf

def vis_dot(V, W):
  return tf.tensordot(V, W, axes = [[1, 2], [0, 1]])

def hid_dot(H, W):
  return tf.tensordot(H, W, axe = [[1], [2]])